<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcTaJWG7PzF3toxaRMB1-JicpqMgJuEXATd0fg&" style="width: 150px;margin-top:30px;" align ="center">

<div style= "font-size: 40px;font-weight:bold; font-family: 'Avenir Next LT Pro', sans-serif;"><center>To Grant or Not to Grant</center></div> <br>

<div style= "font-size: 27px;font-weight:bold;line-height: 1.1; margin-top:40px; font-family: 'Avenir Next LT Pro', sans-serif;"><center>Machine Learning Project 2024/2025</center></div> <br>

   <div style= "font-size: 20px;font-weight:bold; font-family: 'Avenir Next LT Pro', sans-serif;"><center> Group 42:</center></div>
   <div><center> Eden da Silva | 20240740 </center></div>
   <div><center> José Cavaco | 20240513 </center></div>
   <div><center> Luana Rocha | 20240111 </center></div>
   <div><center> Maria Radix | 20240687 </center></div>
   <div><center> Tiago Castilho | 20240489 </center></div>

# Notebook 07: Open Ended Section: Affect of 'Agreement Reached' Prediction on Model Performance

# Index

* [6.0. Imports](#imp)
* [6.1. Open Ended Idea Implementation](#idea)
* [6.2. Dataframe Organization](#org)
* [6.3. Addressing Class Imbalance: SMOTE](#imb)
* [6.4. Predicting Agreement Reached](#agr)
* [6.5. Kaggle Submission](#kaggle)
* [6.6. Conclusion](#conc)

<hr>
<div class="alert alert-block alert-info" style="font-size:12px" id="imp">
    
# **6.0. Imports**
 
</a>

In [4]:
import pandas as pd
import numpy as np

from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, accuracy_score
from sklearn.impute import SimpleImputer

#wrapper methods
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from xgboost import XGBClassifier

# embedded methods
from sklearn.linear_model import LassoCV
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB

from imblearn.over_sampling import SMOTE

import warnings
warnings.filterwarnings('ignore')

In [5]:
#importing standard data
df_train_log_minmax_fs = pd.read_csv('df_train_log_minmax_fs.csv', low_memory=False, index_col = 0)
df_val_log_minmax_fs = pd.read_csv('df_val_log_minmax_fs.csv', low_memory=False, index_col = 0)
df_test_log_minmax_fs = pd.read_csv('df_test_log_minmax_fs.csv', low_memory=False, index_col = 0)

<hr>
<div class="alert alert-block alert-info" style="font-size:12px" id="idea">
    
# **6.1 Open Ended Idea Implementation**
 
</a>

For the open ended section of this project, we decided to explore the usefulness of the 'Agreement Reached' feature that is present in our training data set. The feature is a binary variable that represents whether or not an compensation agreement was reached without the interference of the WCB. As this feature is only present in our training dataset, we decided to create a model that predicted the value of 'Agreement Reached,' and then use these prections in the model that predicts our main target, Claim Injury Type. We tested our four main dataframes on the model predicting Agreement Reached. The model with the highest Macro F1 score was XGBoost with our MinMax Scaled data with some logistic transformations. In this notebook, we calculate and store these predictions on our test data. Then, we will re-run our XGBoost model to predict Claim Injury Type using train and test data that now have Agreement Reached as a column.

<hr>
<div class="alert alert-block alert-info" style="font-size:12px" id="org">
    
# **6.2 DataFrame Organization**
 
</a>

In [9]:
#importing original df
df = pd.read_csv('train_data.csv',low_memory=False, index_col = 'Claim Identifier')

In [10]:
#extracting agreement reached from original df
agreement_reached = df['Agreement Reached']

In [11]:
#align 'Agreemenet Reached' rows with the appropiate rows in the main dfs
agreement_reached_train = agreement_reached.loc[df_train_log_minmax_fs .index]
agreement_reached_val = agreement_reached.loc[df_val_log_minmax_fs .index]

In [12]:
#adding Agreement Reached back into the df we are using
df_train_log_minmax_fs['Agreement Reached'] = agreement_reached_train
df_val_log_minmax_fs['Agreement Reached'] = agreement_reached_val

In [13]:
df_train_log_minmax_fs.head()

,Alternative Dispute Resolution,Attorney/Representative,Birth Year,C-3 Date,COVID-19 Indicator,First Hearing Date,IME-4 Count,Non-Zero Wage,Accident Date in Days,C-2 Date in Days,...,Part of Body_7.0,Part of Body_9.0,Part of Body_10.0,Age Group_Elderly,Age Group_Teen,Accident Season_Spring,Accident Season_Summer,Accident Season_Winter,Claim Injury Type,Agreement Reached
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
5785935,0,1,0.500000,1,0,0,0.057143,1,0.911370,0.666233,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4,0.0
6090033,0,1,0.430556,1,0,1,0.200000,1,0.958170,0.798634,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,0.0
6136197,0,0,0.541667,0,0,0,0.000000,0,0.964410,0.817502,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0
6019545,0,0,0.416667,1,0,0,0.000000,0,0.872429,0.768705,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,0.0
5792247,0,1,0.458333,1,0,1,0.000000,1,0.912295,0.668835,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3,0.0


In [14]:
df_train_log_minmax_fs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 459181 entries, 5785935 to 6027959
Data columns (total 64 columns):
 #   Column                                                    Non-Null Count   Dtype  
---  ------                                                    --------------   -----  
 0   Alternative Dispute Resolution                            459181 non-null  int64  
 1   Attorney/Representative                                   459181 non-null  int64  
 2   Birth Year                                                459181 non-null  float64
 3   C-3 Date                                                  459181 non-null  int64  
 4   COVID-19 Indicator                                        459181 non-null  int64  
 5   First Hearing Date                                        459181 non-null  int64  
 6   IME-4 Count                                               459181 non-null  float64
 7   Non-Zero Wage                                             459181 non-null  int64  
 8   Ac

In [15]:
df_train_log_minmax_fs['Agreement Reached'].isna().sum()

0

In [16]:
df_val_log_minmax_fs.head()

,Alternative Dispute Resolution,Attorney/Representative,Birth Year,C-3 Date,COVID-19 Indicator,First Hearing Date,IME-4 Count,Non-Zero Wage,Accident Date in Days,C-2 Date in Days,...,Part of Body_7.0,Part of Body_9.0,Part of Body_10.0,Age Group_Elderly,Age Group_Teen,Accident Season_Spring,Accident Season_Summer,Accident Season_Winter,Claim Injury Type,Agreement Reached
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
5947783,0,0,0.486111,1,0,0,0.000000,0,0.936908,0.739102,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,3,0.0
6150876,0,1,0.361111,0,0,1,0.028571,1,0.967298,0.824333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,4,0.0
5397365,0,1,0.361111,0,0,0,0.000000,0,0.483245,0.477228,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,4,1.0
6077399,0,1,0.444444,1,0,1,0.142857,1,0.932863,0.796031,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,5,0.0
5945251,0,0,0.708333,0,0,0,0.000000,0,0.937023,0.737150,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2,0.0


In [17]:
df_val_log_minmax_fs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114796 entries, 5947783 to 6076050
Data columns (total 64 columns):
 #   Column                                                    Non-Null Count   Dtype  
---  ------                                                    --------------   -----  
 0   Alternative Dispute Resolution                            114796 non-null  int64  
 1   Attorney/Representative                                   114796 non-null  int64  
 2   Birth Year                                                114796 non-null  float64
 3   C-3 Date                                                  114796 non-null  int64  
 4   COVID-19 Indicator                                        114796 non-null  int64  
 5   First Hearing Date                                        114796 non-null  int64  
 6   IME-4 Count                                               114796 non-null  float64
 7   Non-Zero Wage                                             114796 non-null  int64  
 8   Ac

In [18]:
df_val_log_minmax_fs['Agreement Reached'].isna().sum()

0

In [19]:
df_test_log_minmax_fs.head()

,Alternative Dispute Resolution,Attorney/Representative,Birth Year,C-3 Date,COVID-19 Indicator,First Hearing Date,IME-4 Count,Non-Zero Wage,Accident Date in Days,C-2 Date in Days,...,Part of Body_4.0,Part of Body_5.0,Part of Body_7.0,Part of Body_9.0,Part of Body_10.0,Age Group_Elderly,Age Group_Teen,Accident Season_Spring,Accident Season_Summer,Accident Season_Winter
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
6165911,0,0,0.944444,0,0,0,0.0,1,0.969841,0.832141,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0
6166141,0,0,0.944444,0,0,0,0.0,1,0.965912,0.832141,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6165907,0,0,0.388889,0,0,0,0.0,0,0.970072,0.831490,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6166047,0,0,0.444444,0,0,0,0.0,0,0.970303,0.832141,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
6166102,0,0,0.861111,0,0,0,0.0,0,0.969378,0.831490,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0


In [20]:
df_test_log_minmax_fs.info()

<class 'pandas.core.frame.DataFrame'>
Index: 387975 entries, 6165911 to 6553594
Data columns (total 62 columns):
 #   Column                                                    Non-Null Count   Dtype  
---  ------                                                    --------------   -----  
 0   Alternative Dispute Resolution                            387975 non-null  int64  
 1   Attorney/Representative                                   387975 non-null  int64  
 2   Birth Year                                                387975 non-null  float64
 3   C-3 Date                                                  387975 non-null  int64  
 4   COVID-19 Indicator                                        387975 non-null  int64  
 5   First Hearing Date                                        387975 non-null  int64  
 6   IME-4 Count                                               387975 non-null  float64
 7   Non-Zero Wage                                             387975 non-null  int64  
 8   Ac

In [21]:
#here we set X and y to be from the dataframe we want to examine
X_train = df_train_log_minmax_fs.drop(['Claim Injury Type', 'Agreement Reached'], axis = 1)
X_val = df_val_log_minmax_fs.drop(['Claim Injury Type', 'Agreement Reached'], axis = 1)

y_train = df_train_log_minmax_fs['Claim Injury Type']
y_val = df_val_log_minmax_fs['Claim Injury Type']

ar_train = df_train_log_minmax_fs['Agreement Reached']
ar_val = df_val_log_minmax_fs['Agreement Reached']

In [22]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 459181 entries, 5785935 to 6027959
Data columns (total 62 columns):
 #   Column                                                    Non-Null Count   Dtype  
---  ------                                                    --------------   -----  
 0   Alternative Dispute Resolution                            459181 non-null  int64  
 1   Attorney/Representative                                   459181 non-null  int64  
 2   Birth Year                                                459181 non-null  float64
 3   C-3 Date                                                  459181 non-null  int64  
 4   COVID-19 Indicator                                        459181 non-null  int64  
 5   First Hearing Date                                        459181 non-null  int64  
 6   IME-4 Count                                               459181 non-null  float64
 7   Non-Zero Wage                                             459181 non-null  int64  
 8   Ac

In [23]:
X_val.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114796 entries, 5947783 to 6076050
Data columns (total 62 columns):
 #   Column                                                    Non-Null Count   Dtype  
---  ------                                                    --------------   -----  
 0   Alternative Dispute Resolution                            114796 non-null  int64  
 1   Attorney/Representative                                   114796 non-null  int64  
 2   Birth Year                                                114796 non-null  float64
 3   C-3 Date                                                  114796 non-null  int64  
 4   COVID-19 Indicator                                        114796 non-null  int64  
 5   First Hearing Date                                        114796 non-null  int64  
 6   IME-4 Count                                               114796 non-null  float64
 7   Non-Zero Wage                                             114796 non-null  int64  
 8   Ac

<hr>
<div class="alert alert-block alert-info" style="font-size:12px" id="imb">
    
# **6.3 Addressing Class Imbalance: SMOTE**
 
</a>

As in our modeling notebook, we will use Synthetic Minority Oversampling Technique (SMOTE) to solve the class imbalance that is present in our target feature, Claim Injury Type. From previous feature exploration, we know the categories that represent extreme case injuries (categories 7 and 8) are very rare. SMOTE works by analyzing the categories with few values and generating similar points. The addition of these 'synthetic' points helps to balance the dataset, making it easier for models to predict the more rare categories.

Source: https://www.analyticsvidhya.com/blog/2020/10/overcoming-class-imbalance-using-smote-techniques/

In [26]:
#initializing SMOTE for Agreement Reached
smote_ar = SMOTE(random_state=2)

X_ar_train_resampled, ar_train_resampled = smote_ar.fit_resample(X_train, ar_train)

<hr>
<div class="alert alert-block alert-info" style="font-size:12px" id="agr">
    
# **6.4 Predicting Agreement Reached**
 
</a>

In [28]:
## Agreement Reached

xgb_ar = XGBClassifier()

xgb_ar.fit(X_ar_train_resampled, ar_train_resampled)

# Predict and evaluate
ar_val_pred = xgb_ar.predict(X_val)
ar_train_pred = xgb_ar.predict(X_train)

print("Accuracy Train Score:", accuracy_score(ar_train, ar_train_pred))
print("Accuracy Val Score:", accuracy_score(ar_val, ar_val_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(ar_val, ar_val_pred))
print("\nClassification Report:")
print(classification_report(ar_val, ar_val_pred))

Accuracy Train Score: 0.9561393001888144
Accuracy Val Score: 0.9520976340639047

Confusion Matrix:
[[107937   1411]
 [  4088   1360]]

Classification Report:
              precision    recall  f1-score   support

         0.0       0.96      0.99      0.98    109348
         1.0       0.49      0.25      0.33      5448

    accuracy                           0.95    114796
   macro avg       0.73      0.62      0.65    114796
weighted avg       0.94      0.95      0.94    114796



In [29]:
df_test_agr_log_minmax = df_test_log_minmax_fs.copy()

In [30]:
df_test_agr_log_minmax['Agreement Reached'] = xgb_ar.predict(df_test_agr_log_minmax)

In [31]:
df_test_agr_log_minmax.head()

,Alternative Dispute Resolution,Attorney/Representative,Birth Year,C-3 Date,COVID-19 Indicator,First Hearing Date,IME-4 Count,Non-Zero Wage,Accident Date in Days,C-2 Date in Days,...,Part of Body_5.0,Part of Body_7.0,Part of Body_9.0,Part of Body_10.0,Age Group_Elderly,Age Group_Teen,Accident Season_Spring,Accident Season_Summer,Accident Season_Winter,Agreement Reached
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
6165911,0,0,0.944444,0,0,0,0.0,1,0.969841,0.832141,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0
6166141,0,0,0.944444,0,0,0,0.0,1,0.965912,0.832141,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
6165907,0,0,0.388889,0,0,0,0.0,0,0.970072,0.831490,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
6166047,0,0,0.444444,0,0,0,0.0,0,0.970303,0.832141,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
6166102,0,0,0.861111,0,0,0,0.0,0,0.969378,0.831490,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0


<hr>
<div class="alert alert-block alert-info" style="font-size:12px" id="kaggle">
    
# **6.5 Predicting Claim Injury Type with Agreement Reached**
 
</a>

In [33]:
#adjusting our data to now point to the test df that contains predicrions for Agreement Reached
X_train = df_train_log_minmax_fs.drop(['Claim Injury Type'], axis = 1)
X_val = df_val_log_minmax_fs.drop(['Claim Injury Type'], axis = 1)

y_train = df_train_log_minmax_fs['Claim Injury Type']
y_val = df_val_log_minmax_fs['Claim Injury Type']

In [34]:
#checking to make sure Agreement Reached is now included in train data
X_train.head()

,Alternative Dispute Resolution,Attorney/Representative,Birth Year,C-3 Date,COVID-19 Indicator,First Hearing Date,IME-4 Count,Non-Zero Wage,Accident Date in Days,C-2 Date in Days,...,Part of Body_5.0,Part of Body_7.0,Part of Body_9.0,Part of Body_10.0,Age Group_Elderly,Age Group_Teen,Accident Season_Spring,Accident Season_Summer,Accident Season_Winter,Agreement Reached
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
5785935,0,1,0.500000,1,0,0,0.057143,1,0.911370,0.666233,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
6090033,0,1,0.430556,1,0,1,0.200000,1,0.958170,0.798634,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6136197,0,0,0.541667,0,0,0,0.000000,0,0.964410,0.817502,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6019545,0,0,0.416667,1,0,0,0.000000,0,0.872429,0.768705,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5792247,0,1,0.458333,1,0,1,0.000000,1,0.912295,0.668835,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [35]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 459181 entries, 5785935 to 6027959
Data columns (total 63 columns):
 #   Column                                                    Non-Null Count   Dtype  
---  ------                                                    --------------   -----  
 0   Alternative Dispute Resolution                            459181 non-null  int64  
 1   Attorney/Representative                                   459181 non-null  int64  
 2   Birth Year                                                459181 non-null  float64
 3   C-3 Date                                                  459181 non-null  int64  
 4   COVID-19 Indicator                                        459181 non-null  int64  
 5   First Hearing Date                                        459181 non-null  int64  
 6   IME-4 Count                                               459181 non-null  float64
 7   Non-Zero Wage                                             459181 non-null  int64  
 8   Ac

In [36]:
X_val.head()

,Alternative Dispute Resolution,Attorney/Representative,Birth Year,C-3 Date,COVID-19 Indicator,First Hearing Date,IME-4 Count,Non-Zero Wage,Accident Date in Days,C-2 Date in Days,...,Part of Body_5.0,Part of Body_7.0,Part of Body_9.0,Part of Body_10.0,Age Group_Elderly,Age Group_Teen,Accident Season_Spring,Accident Season_Summer,Accident Season_Winter,Agreement Reached
Claim Identifier,,,,,,,,,,,,,,,,,,,,,
5947783,0,0,0.486111,1,0,0,0.000000,0,0.936908,0.739102,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
6150876,0,1,0.361111,0,0,1,0.028571,1,0.967298,0.824333,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5397365,0,1,0.361111,0,0,0,0.000000,0,0.483245,0.477228,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0
6077399,0,1,0.444444,1,0,1,0.142857,1,0.932863,0.796031,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
5945251,0,0,0.708333,0,0,0,0.000000,0,0.937023,0.737150,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [37]:
X_val.info()

<class 'pandas.core.frame.DataFrame'>
Index: 114796 entries, 5947783 to 6076050
Data columns (total 63 columns):
 #   Column                                                    Non-Null Count   Dtype  
---  ------                                                    --------------   -----  
 0   Alternative Dispute Resolution                            114796 non-null  int64  
 1   Attorney/Representative                                   114796 non-null  int64  
 2   Birth Year                                                114796 non-null  float64
 3   C-3 Date                                                  114796 non-null  int64  
 4   COVID-19 Indicator                                        114796 non-null  int64  
 5   First Hearing Date                                        114796 non-null  int64  
 6   IME-4 Count                                               114796 non-null  float64
 7   Non-Zero Wage                                             114796 non-null  int64  
 8   Ac

In [38]:
# Initialize SMOTE for X and y
smote = SMOTE(random_state=1)

# Resample the training data
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)


In [39]:
#initializing XGBoost on data that contains Agreement Reached predictions
## XGB expects y to start from 0 so we changed y_train and y_val
y_train_xgb_res = y_train_resampled - 1
y_train_xgb = y_train - 1
y_val_xgb = y_val - 1

# Initialize and train XGBoost
xgb = XGBClassifier()                   

xgb.fit(X_train_resampled, y_train_xgb_res)
# Predict and evaluate
y_val_pred = xgb.predict(X_val)
y_train_pred = xgb.predict(X_train)

print("Accuracy Train Score:", accuracy_score(y_train_xgb, y_train_pred))
print("Accuracy Val Score:", accuracy_score(y_val_xgb, y_val_pred))
print("\nConfusion Matrix:")
print(confusion_matrix(y_val_xgb, y_val_pred))
print("\nClassification Report:")
print(classification_report(y_val_xgb, y_val_pred))

Accuracy Train Score: 0.8038899693149325
Accuracy Val Score: 0.7985034321753371

Confusion Matrix:
[[ 1591   807    65    18    11     0     0     4]
 [ 1148 56010   777   188    52     3     1    29]
 [   56  7318  1723  3742   816   103     1    21]
 [    3   553   657 24834  2712   886    10    46]
 [    1    32   128  2017  7224   250     3     1]
 [    0     1    16   438   163   221     3     0]
 [    0     0     3     7     3     5     0     1]
 [    1    14     5    12     0     0     0    62]]

Classification Report:
              precision    recall  f1-score   support

           0       0.57      0.64      0.60      2496
           1       0.87      0.96      0.91     58208
           2       0.51      0.13      0.20     13780
           3       0.79      0.84      0.81     29701
           4       0.66      0.75      0.70      9656
           5       0.15      0.26      0.19       842
           6       0.00      0.00      0.00        19
           7       0.38      0.66  

<hr>
<div class="alert alert-block alert-info" style="font-size:12px" id="kaggle">
    
# **6.6 Kaggle Submission**
 
</a>

In [41]:
df_test_agr_log_minmax['Claim Injury Type'] = xgb.predict(df_test_agr_log_minmax)

In [42]:
df_test_agr_log_minmax['Claim Injury Type'] = df_test_agr_log_minmax['Claim Injury Type']+1


In [43]:
kaggle_submission = df_test_agr_log_minmax[['Claim Injury Type']].copy()

In [44]:
print(kaggle_submission)

                  Claim Injury Type
Claim Identifier                   
6165911                           2
6166141                           2
6165907                           2
6166047                           2
6166102                           2
...                             ...
6553137                           1
6553119                           1
6553542                           1
6553455                           1
6553594                           1

[387975 rows x 1 columns]


In [45]:
#creating a dictionary so that we can access Claim Injury Type descriptions when we make our predictions with the model
claim_injury_type_dict = {1: '1. CANCELLED', 2: '2. NON-COMP', 3: '3. MED ONLY', 4: '4. TEMPORARY', 5: '5. PPD SCH LOSS',
                          6: '6. PPD NSL', 7: '7. PTD', 8: '8. DEATH'}

In [46]:
kaggle_submission['Claim Injury Type'] = kaggle_submission['Claim Injury Type'].map(claim_injury_type_dict)

In [47]:
print(kaggle_submission)

                 Claim Injury Type
Claim Identifier                  
6165911                2. NON-COMP
6166141                2. NON-COMP
6165907                2. NON-COMP
6166047                2. NON-COMP
6166102                2. NON-COMP
...                            ...
6553137               1. CANCELLED
6553119               1. CANCELLED
6553542               1. CANCELLED
6553455               1. CANCELLED
6553594               1. CANCELLED

[387975 rows x 1 columns]


In [48]:
# #export test data predictions
kaggle_submission.to_csv('ClaimInjuryType_predictions_with_agr_minmax.csv')

<hr>
<div class="alert alert-block alert-info" style="font-size:12px" id="conc">
    
# **6.6 Conclusion**
 
</a>

Conclusion: Adding Agreement Reached did increase our score slightly (a margin of 0.02 in Macro F1 on Validation data, and a margin of 0.004 on test data in Kaggle). Overall, however, it did not have a significant impact on our model’s performance. If this were a real-world scenario, our model still not be accurate enough to be considered viable for use in making real predictions. Perhaps if we had put more consideration on Agreement Reached when making our preprocessing decisions, these results would be different. We had our focus set on Claim Injury Type, and did not analyze any individual pairwise relationships between Agreement Reached and other features in the data.  